In [16]:
import os
from os import walk

import pandas as pd
import numpy as np
import sys
import re

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from collections import Counter

%matplotlib inline

pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 1500)
pd.set_option('max_colwidth', 400)

# Load result dataframes

In [17]:
#Overall result dataframe:

coverage_per_sample = pd.read_csv('result_data/sample_depth.csv', index_col=[0])
coverage_per_sample.head()

,mean_depth
HG00384,85.06
NA19448,74.82
HG01498,96.26
HG00705,99.01
HG00693,95.60


# Make downsampling dataframe

In [18]:
coverage_per_sample['200X'] = 200 / coverage_per_sample['mean_depth'] 
coverage_per_sample['146X'] = 146 / coverage_per_sample['mean_depth'] 
coverage_per_sample['100X'] = 100 / coverage_per_sample['mean_depth'] 
coverage_per_sample['75X'] = 75 / coverage_per_sample['mean_depth'] 
coverage_per_sample['50X'] = 50 / coverage_per_sample['mean_depth'] 
coverage_per_sample['20X'] = 20 / coverage_per_sample['mean_depth']
coverage_per_sample['10X'] = 10 / coverage_per_sample['mean_depth']
coverage_per_sample['5X'] = 5 / coverage_per_sample['mean_depth'] 
coverage_per_sample['2X'] = 2 / coverage_per_sample['mean_depth']
coverage_per_sample['1X'] = 1 / coverage_per_sample['mean_depth']

In [19]:
df = coverage_per_sample[coverage_per_sample['mean_depth'] > 100]
print(len(df.sort_values('mean_depth', ascending=False)))
with open('result_data/sample_id_min_100X.txt', 'w') as outfile:
    for sample_id in list(df.index):
        outfile.write(f'{sample_id}\n')

df = coverage_per_sample[coverage_per_sample['mean_depth'] > 125]
print(len(df.sort_values('mean_depth', ascending=False)))
with open('result_data/sample_id_min_125X.txt', 'w') as outfile:
    for sample_id in list(df.index):
        outfile.write(f'{sample_id}\n')

df = coverage_per_sample[coverage_per_sample['mean_depth'] > 150]
print(len(df.sort_values('mean_depth', ascending=False)))
with open('result_data/sample_id_min_150X.txt', 'w') as outfile:
    for sample_id in list(df.index):
        outfile.write(f'{sample_id}\n')


230
79
47


In [20]:
coverage_per_sample.sort_values('mean_depth', ascending=False).head()

,mean_depth,200X,146X,100X,75X,50X,20X,10X,5X,2X,1X
NA20313,456.34,0.438270,0.319937,0.219135,0.164351,0.109567,0.043827,0.021913,0.010957,0.004383,0.002191
HG00731,345.07,0.579593,0.423103,0.289796,0.217347,0.144898,0.057959,0.028980,0.014490,0.005796,0.002898
HG01873,305.59,0.654472,0.477764,0.327236,0.245427,0.163618,0.065447,0.032724,0.016362,0.006545,0.003272
HG00732,298.06,0.671006,0.489834,0.335503,0.251627,0.167751,0.067101,0.033550,0.016775,0.006710,0.003355
HG01872,280.99,0.711769,0.519591,0.355885,0.266913,0.177942,0.071177,0.035588,0.017794,0.007118,0.003559


In [21]:
high_coverage_samples = list(coverage_per_sample[coverage_per_sample['mean_depth'] > 110].index)

In [22]:
print(len(coverage_per_sample[coverage_per_sample['200X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['146X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['100X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['75X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['50X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['20X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['10X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['5X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['2X'] < 1]))
print(len(coverage_per_sample[coverage_per_sample['1X'] < 1]))

13
50
230
543
779
829
829
829
829
829


In [23]:
print(len(coverage_per_sample[coverage_per_sample['100X'] < 1]))

230


In [24]:
np.mean(list(coverage_per_sample['mean_depth']))

89.95799758745476

In [25]:
coverage_per_sample_100 = coverage_per_sample[coverage_per_sample['mean_depth'] > 100]

coverage_100_id_list = list(coverage_per_sample_100.index)

converage_100_multipliers = list(coverage_per_sample_100['100X'])
converage_50_multipliers = list(coverage_per_sample_100['50X'])
converage_20_multipliers = list(coverage_per_sample_100['20X'])
converage_10_multipliers = list(coverage_per_sample_100['10X'])
converage_5_multipliers = list(coverage_per_sample_100['5X'])
converage_2_multipliers = list(coverage_per_sample_100['2X'])
converage_1_multipliers = list(coverage_per_sample_100['1X'])

## Write configfiles with multipliers for downsampling

In [26]:
snakemake_folder_path = '../Snakemake/'

configfile_100 = snakemake_folder_path + "config_100X.yaml"
configfile_75 = snakemake_folder_path + 'config_75X.yaml'
configfile_50 = snakemake_folder_path + 'config_50X.yaml'
configfile_20 = snakemake_folder_path + 'config_20X.yaml'
configfile_10 = snakemake_folder_path + 'config_10X.yaml'
configfile_5 = snakemake_folder_path + 'config_5X.yaml'
configfile_2 = snakemake_folder_path + 'config_2X.yaml'
configfile_1 = snakemake_folder_path + 'config_1X.yaml'

with open(configfile_100, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '100X'])[2:] + "\"" + '\n')

with open(configfile_75, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '75X'])[2:] + "\"" + '\n')
        
with open(configfile_50, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '50X'])[2:] + "\"" + '\n')

with open(configfile_20, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '20X'])[2:] + "\"" + '\n')

with open(configfile_10, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '10X'])[2:] + "\"" + '\n')

with open(configfile_5, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '5X'])[2:] + "\"" + '\n')

with open(configfile_2, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '2X'])[2:] + "\"" + '\n')

with open(configfile_1, 'w') as outfile:
    outfile.write('sample_downsample_fraction:\n')
    for sample_id in coverage_100_id_list:
        outfile.write("  " + sample_id + ": " +  "\"" + str(coverage_per_sample_100.loc[sample_id, '1X'])[2:] + "\"" + '\n') 

# Write Configfile for Gargammel (aDNA) project

In [27]:
print(len(coverage_per_sample[coverage_per_sample['146X'] < 1]))

50


In [28]:
coverage_per_sample_146 = coverage_per_sample[coverage_per_sample['mean_depth'] > 146]

coverage_146_id_list = list(coverage_per_sample_146.index)

In [29]:
with open('../aDNA/config.yaml', 'w') as outfile:
    outfile.write('sample_name:\n')
    for sample_id in coverage_146_id_list:
        outfile.write("  " + sample_id + '\n')
